In [ ]:
import pandas as pd
import numpy as np
import re
import time
from datetime import timedelta
from typing import List, Dict, Tuple
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import os
from dotenv import load_dotenv

load_dotenv()

sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

In [5]:
from datasets import load_dataset

ds = load_dataset("har1/MTS_Dialogue-Clinical_Note")

In [ ]:

print("Dataset structure:")
print(ds)
print("\nKeys in dataset:", ds.keys())
print("\nTrain set size:", len(ds['train']))

# Display first example
print("\n" + "="*80)
print("SAMPLE DIALOGUE:")
print("="*80)
sample = ds['train'][0]
for key, value in sample.items():
    print(f"\n{key.upper()}:")
    print("-" * 40)
    print(value if len(str(value)) < 500 else str(value)[:500] + "...")

Dataset structure:
DatasetDict({
    train: Dataset({
        features: ['ID', 'section_header', 'section_text', 'dialogue'],
        num_rows: 1301
    })
})

Keys in dataset: dict_keys(['train'])

Train set size: 1301

SAMPLE DIALOGUE:

ID:
----------------------------------------
0

SECTION_HEADER:
----------------------------------------
GENHX

SECTION_TEXT:
----------------------------------------
Symptoms: no fever, no chills, no cough, no congestion, no nausea, no vomiting, no chest pain, no chest pressure.
Diagnosis: hypertension, osteoarthritis, osteoporosis, hypothyroidism, allergic rhinitis, kidney stones
History of Patient: 76-year-old white female, presents to the clinic today originally for hypertension and a med check, followed by Dr. Kumar, issues stable
Plan of Action: N/A

DIALOGUE:
----------------------------------------
Doctor: What brings you back into the clinic today, miss? 
Patient: I came in for a refill of my blood pressure medicine. 
Doctor: It looks like Do

In [ ]:

from groq import Groq
from typing import List
import json

class GroqAPILabeler:
   
    
    def __init__(self, api_key: str, model: str = "llama-3.1-70b-versatile"):
        
        self.client = Groq(api_key=api_key)
        self.model = model
        
        print(f"✓ Initialized Groq API with model: {model}")
        print(f"⚡ Expected speed: ~0.3-0.5s per sample")
        
        self.event_categories = [
            'medication_error', 'allergic_reaction', 'symptom_worsening',
            'side_effects', 'non_compliance', 'emergency', 'infection',
            'no_adverse_event'
        ]
    
    def create_prompt(self, dialogue: str, clinical_note: str) -> str:
        """Create efficient prompt for Groq"""
        dialogue_short = dialogue[:800] if len(dialogue) > 800 else dialogue
        note_short = clinical_note[:400] if len(clinical_note) > 400 else clinical_note
        
        prompt = f"""Analyze this patient-nurse conversation and identify adverse medical events.

ADVERSE EVENT TYPES:
- medication_error: Wrong dose, missed medication, incorrect prescription
- allergic_reaction: Allergic response, rash, hives, breathing issues
- symptom_worsening: Patient's condition deteriorating
- side_effects: Medication side effects (nausea, dizziness, etc.)
- non_compliance: Not taking medication as prescribed
- emergency: Urgent medical situation requiring immediate care
- infection: Signs of infection, fever, inflammation
- no_adverse_event: Routine visit with no issues

CRITICAL RULES:
1. Only identify events if the PATIENT is actually experiencing them
2. If patient answers "No" to doctor's symptom questions → label as "no_adverse_event"
3. Ignore symptoms mentioned in questions unless patient confirms them
4. Focus on patient's actual responses and confirmed findings

CONVERSATION:
{dialogue_short}

CLINICAL NOTE:
{note_short}

Return ONLY a valid JSON array of applicable labels. Examples:
- Routine visit: ["no_adverse_event"]
- Patient has side effects: ["side_effects"]
- Multiple issues: ["medication_error", "side_effects"]

JSON array:"""
        
        return prompt
    
    def extract_labels_groq(self, dialogue: str, clinical_note: str) -> List[str]:
        """Extract labels using Groq API"""
        try:
            prompt = self.create_prompt(dialogue, clinical_note)
            
            response = self.client.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": "You are a medical AI expert at identifying adverse events. Always return valid JSON arrays only."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.1,
                max_tokens=100
            )
            
            content = response.choices[0].message.content.strip()
            
            # Extract JSON
            if "[" in content and "]" in content:
                start = content.index("[")
                end = content.rindex("]") + 1
                json_str = content[start:end]
                labels = json.loads(json_str)
                
                # Validate
                labels = [l for l in labels if l in self.event_categories]
                return labels if labels else ['no_adverse_event']
            
        except Exception as e:
            print(f"Groq API Error: {e}")
        
        return ['no_adverse_event']
    
    def create_risk_score(self, labels: List[str]) -> float:
        """Assign risk score"""
        severity_weights = {
            'emergency': 1.0,
            'allergic_reaction': 0.9,
            'infection': 0.85,
            'medication_error': 0.8,
            'symptom_worsening': 0.7,
            'side_effects': 0.5,
            'non_compliance': 0.4,
            'no_adverse_event': 0.0
        }
        
        if not labels or labels == ['no_adverse_event']:
            return 0.0
        
        scores = [severity_weights.get(label, 0.5) for label in labels]
        base_score = max(scores)
        
        if len(labels) > 1:
            base_score = min(1.0, base_score + 0.1)
        
        return round(base_score, 2)
    
    def get_risk_level(self, risk_score: float) -> str:
        """Convert risk score to category"""
        if risk_score >= 0.8:
            return "CRITICAL"
        elif risk_score >= 0.6:
            return "HIGH"
        elif risk_score >= 0.3:
            return "MODERATE"
        elif risk_score > 0:
            return "LOW"
        else:
            return "NONE"

print("✓ GroqAPILabeler class created")

✓ GroqAPILabeler class created


In [ ]:
def process_dataset_groq(dataset, groq_labeler: GroqAPILabeler, max_samples: int = None) -> pd.DataFrame:
    
    processed_data = []
    num_samples = len(dataset) if max_samples is None else min(max_samples, len(dataset))
    
    print(f"\n{'='*80}")
    print(f"⚡ GROQ API PROCESSING - ULTRA FAST")
    print(f"{'='*80}")
    print(f"Total samples: {num_samples}")
    print(f"Model: {groq_labeler.model}")
    print(f"Expected time: ~{int(num_samples * 0.4 / 60)} minutes")
    print(f"{'='*80}\n")
    
    # Track timing
    start_time = time.time()
    sample_times = []
    
    pbar = tqdm(range(num_samples), desc="⚡ Processing", unit="sample")
    
    for idx in pbar:
        sample_start = time.time()
        
        try:
            item = dataset[idx]
            
            
            dialogue = item.get('dialogue', item.get('Dialogue', item.get('conversation', '')))
            clinical_note = item.get('section_text', item.get('clinical_note', item.get('note', '')))
            
            # Generate labels using Groq API
            labels = groq_labeler.extract_labels_groq(dialogue, clinical_note)
            risk_score = groq_labeler.create_risk_score(labels)
            risk_level = groq_labeler.get_risk_level(risk_score)
            
            processed_data.append({
                'id': idx,
                'dialogue': dialogue,
                'clinical_note': clinical_note,
                'labels': labels,
                'num_events': len([l for l in labels if l != 'no_adverse_event']),
                'risk_score': risk_score,
                'risk_level': risk_level,
                'has_adverse_event': 'no_adverse_event' not in labels
            })
            
            # Track timing
            sample_time = time.time() - sample_start
            sample_times.append(sample_time)
            
            # Update progress bar with timing info
            if len(sample_times) > 0:
                avg_time = np.mean(sample_times)
                remaining = num_samples - (idx + 1)
                eta_seconds = remaining * avg_time
                eta = timedelta(seconds=int(eta_seconds))
                
                pbar.set_postfix({
                    'speed': f'{avg_time:.2f}s',
                    'ETA': str(eta),
                    'events': sum([d['has_adverse_event'] for d in processed_data])
                })
            
        except Exception as e:
            print(f"\n⚠️  Error processing sample {idx}: {e}")
            # Fallback
            processed_data.append({
                'id': idx,
                'dialogue': dialogue if 'dialogue' in locals() else '',
                'clinical_note': clinical_note if 'clinical_note' in locals() else '',
                'labels': ['no_adverse_event'],
                'num_events': 0,
                'risk_score': 0.0,
                'risk_level': 'NONE',
                'has_adverse_event': False
            })
    
    pbar.close()
    
    # Final timing report
    total_time = time.time() - start_time
    avg_time_per_sample = total_time / num_samples if num_samples > 0 else 0
    
    print(f"\n{'='*80}")
    print(f"✅ PROCESSING COMPLETE")
    print(f"{'='*80}")
    print(f"⏱️  Total time: {timedelta(seconds=int(total_time))}")
    print(f"⚡ Average speed: {avg_time_per_sample:.2f}s per sample")
    print(f"📊 Samples processed: {len(processed_data)}")
    print(f"🚨 Adverse events found: {sum([d['has_adverse_event'] for d in processed_data])}")
    print(f"💰 Estimated cost: ~$0.00 (Groq is nearly free)")
    print(f"{'='*80}\n")
    
    df = pd.DataFrame(processed_data)
    return df

print("✓ Groq processing function created")

✓ Groq processing function created


In [ ]:

api_key = os.getenv("GROQ_API_KEY")
if not api_key:
    raise ValueError("GROQ_API_KEY not found. Create a .env file with your API key.")

groq_labeler = GroqAPILabeler(
    api_key=api_key,
    model="llama-3.1-8b-instant"
)

print(f"✓ Initialized Groq API with model: {groq_labeler.model} ⚡")
print(f"Expected speed: ~0.3-0.5s per sample")

✓ Initialized Groq API with model: llama-3.1-8b-instant
⚡ Expected speed: ~0.3-0.5s per sample
✓ Initialized Groq API with model: llama-3.1-8b-instant ⚡
Expected speed: ~0.3-0.5s per sample


In [ ]:

if 'ds' not in locals():
    from datasets import load_dataset
    ds = load_dataset("har1/MTS_Dialogue-Clinical_Note")
    print(f"✓ Dataset loaded: {len(ds['train'])} samples")
else:
    print(f"✓ Dataset already loaded: {len(ds['train'])} samples")

f:\project\hackathon\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✓ Dataset loaded: 1301 samples


In [ ]:


df_groq_labeled = process_dataset_groq(
    ds['train'], 
    groq_labeler, 
    max_samples=None  # Process ALL samples (change to smaller number for testing)
)

print("\n✅ Labeling complete! Let's analyze the results...")


⚡ GROQ API PROCESSING - ULTRA FAST
Total samples: 1301
Model: llama-3.1-8b-instant
Expected time: ~8 minutes



⚡ Processing:   1%|▏         | 17/1301 [00:29<1:39:29,  4.65s/sample, speed=1.74s, ETA=0:37:09, events=5]

Groq API Error: Expecting ',' delimiter: line 3 column 16 (char 40)


⚡ Processing:   9%|▉         | 115/1301 [07:51<1:27:59,  4.45s/sample, speed=4.10s, ETA=1:20:58, events=34]

Groq API Error: Expecting ',' delimiter: line 3 column 16 (char 40)


...

Groq API Error: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.1-8b-instant` in organization `org_01k7st5283f8stp8xgcvhv0dne` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 499999, Requested 370. Please try again in 1m3.7632s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}
Groq API Error: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.1-8b-instant` in organization `org_01k7st5283f8stp8xgcvhv0dne` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 499998, Requested 520. Please try again in 1m29.5104s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}


⚡ Processing: 100%|█████████▉| 1300/1301 [1:41:37<00:02,  2.85s/sample, speed=4.69s, ETA=0:00:04, events=384]

Groq API Error: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.1-8b-instant` in organization `org_01k7st5283f8stp8xgcvhv0dne` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 499997, Requested 392. Please try again in 1m7.2192s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}
Groq API Error: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.1-8b-instant` in organization `org_01k7st5283f8stp8xgcvhv0dne` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 499997, Requested 360. Please try again in 1m1.6896s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}


⚡ Processing: 100%|██████████| 1301/1301 [1:41:37<00:00,  4.69s/sample, speed=4.69s, ETA=0:00:00, events=384]

Groq API Error: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.1-8b-instant` in organization `org_01k7st5283f8stp8xgcvhv0dne` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 499996, Requested 498. Please try again in 1m25.363199999s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

✅ PROCESSING COMPLETE
⏱️  Total time: 1:41:37
⚡ Average speed: 4.69s per sample
📊 Samples processed: 1301
🚨 Adverse events found: 384
💰 Estimated cost: ~$0.00 (Groq is nearly free)


✅ Labeling complete! Let's analyze the results...


In [17]:
# Analyze results
print("\n" + "="*80)
print("📊 GROQ LABELING RESULTS")
print("="*80)

# Basic statistics
print(f"\nTotal samples: {len(df_groq_labeled)}")
print(f"Adverse events found: {df_groq_labeled['has_adverse_event'].sum()}")
print(f"No adverse events: {(~df_groq_labeled['has_adverse_event']).sum()}")
print(f"Average risk score: {df_groq_labeled['risk_score'].mean():.3f}")

# Label distribution
all_groq_labels = []
for labels_list in df_groq_labeled['labels']:
    all_groq_labels.extend(labels_list)

groq_label_counts = Counter(all_groq_labels)

print("\n" + "-"*80)
print("ADVERSE EVENT DISTRIBUTION:")
print("-"*80)
for label, count in groq_label_counts.most_common():
    percentage = (count / len(df_groq_labeled)) * 100
    print(f"  {label:25s}: {count:4d} ({percentage:5.1f}%)")

# Risk level distribution
print("\n" + "-"*80)
print("RISK LEVEL DISTRIBUTION:")
print("-"*80)
risk_dist = df_groq_labeled['risk_level'].value_counts()
for level in ['CRITICAL', 'HIGH', 'MODERATE', 'LOW', 'NONE']:
    count = risk_dist.get(level, 0)
    percentage = (count / len(df_groq_labeled)) * 100
    print(f"  {level:10s}: {count:4d} ({percentage:5.1f}%)")

print("\n" + "="*80)


📊 GROQ LABELING RESULTS

Total samples: 1301
Adverse events found: 384
No adverse events: 917
Average risk score: 0.286

--------------------------------------------------------------------------------
ADVERSE EVENT DISTRIBUTION:
--------------------------------------------------------------------------------
  no_adverse_event         :  917 ( 70.5%)
  symptom_worsening        :  335 ( 25.7%)
  infection                :  223 ( 17.1%)
  side_effects             :  179 ( 13.8%)
  medication_error         :  151 ( 11.6%)
  allergic_reaction        :   99 (  7.6%)
  emergency                :   83 (  6.4%)
  non_compliance           :   71 (  5.5%)

--------------------------------------------------------------------------------
RISK LEVEL DISTRIBUTION:
--------------------------------------------------------------------------------
  CRITICAL  :  388 ( 29.8%)
  HIGH      :   11 (  0.8%)
  MODERATE  :    7 (  0.5%)
  LOW       :    0 (  0.0%)
  NONE      :  895 ( 68.8%)



In [ ]:

print("\n" + "="*80)
print("🔍 VERIFICATION: Testing previously problematic sample")
print("="*80)

test_sample = df_groq_labeled.iloc[0]

print(f"\nDialogue (first 300 chars):")
print(f"{test_sample['dialogue'][:300]}...")

print(f"\n✅ Groq Labels: {test_sample['labels']}")
print(f"   Risk Score: {test_sample['risk_score']}")
print(f"   Risk Level: {test_sample['risk_level']}")

print(f"\n❌ Original Pattern-based (for comparison):")
print(f"   Labels: ['emergency', 'infection', 'side_effects']")
print(f"   Risk Score: 1.0")
print(f"   Risk Level: CRITICAL")

if test_sample['labels'] == ['no_adverse_event']:
    print(f"\n✅ SUCCESS: Groq correctly identifies routine visit!")
else:
    print(f"\n⚠️  Groq found: {test_sample['labels']}")


🔍 VERIFICATION: Testing previously problematic sample

Dialogue (first 300 chars):
Doctor: What brings you back into the clinic today, miss? 
Patient: I came in for a refill of my blood pressure medicine. 
Doctor: It looks like Doctor Kumar followed up with you last time regarding your hypertension, osteoarthritis, osteoporosis, hypothyroidism, allergic rhinitis and kidney stones....

✅ Groq Labels: ['no_adverse_event']
   Risk Score: 0.0
   Risk Level: NONE

❌ Original Pattern-based (for comparison):
   Labels: ['emergency', 'infection', 'side_effects']
   Risk Score: 1.0
   Risk Level: CRITICAL

✅ SUCCESS: Groq correctly identifies routine visit!


In [ ]:

output_file = 'labeled_dialogues_groq.csv'
df_groq_labeled.to_csv(output_file, index=False)
print(f"✓ Groq-labeled dataset saved to: {output_file}")

# Also save expanded version
df_groq_expanded = df_groq_labeled.copy()
df_groq_expanded['labels'] = df_groq_expanded['labels'].apply(lambda x: '|'.join(x))
df_groq_expanded.to_csv('labeled_dialogues_groq_expanded.csv', index=False)
print(f"✓ Expanded version saved to: labeled_dialogues_groq_expanded.csv")

print(f"\n📊 Final Dataset Info:")
print(f"   Shape: {df_groq_labeled.shape}")
print(f"   Columns: {list(df_groq_labeled.columns)}")
print(f"\n✅ All done! Dataset ready for model training.")

✓ Groq-labeled dataset saved to: labeled_dialogues_groq.csv
✓ Expanded version saved to: labeled_dialogues_groq_expanded.csv

📊 Final Dataset Info:
   Shape: (1301, 8)
   Columns: ['id', 'dialogue', 'clinical_note', 'labels', 'num_events', 'risk_score', 'risk_level', 'has_adverse_event']

✅ All done! Dataset ready for model training.
